In [ ]:
# リスト 2.1.10
# PDFファイルの読み込み（Apache Tika を用いてリモート PDF の本文とメタデータを抽出する最小例）
# ------------------------------------------------------------------------------------------------
# 目的:
#   - Python バインディング（python-tika）の `parser.from_file` を用いて、指定した PDF の解析結果を得る。
#
# 背景（理論・実務ポイント）:
#   - Apache Tika は多種多様な文書形式から「本文テキスト（content）」と「メタデータ（metadata）」を抽出する統合パーサ。
#   - python-tika は内部で Java の Tika-App/Tika-Server を起動し、HTTP 経由で解析するラッパである。
#     → 初回呼び出し時に JAR のダウンロードや Java VM の起動が行われるため、環境に JRE/JDK（一般に Java 8+）が必要。
#   - `parser.from_file()` はローカルパスだけでなく URL 文字列も受け付ける。
#     → この例では GitHub Raw の PDF を直接解析対象にしており、Tika 側がリモート取得→解析を行う。
#   - 返り値は辞書（dict）で、主に `parsed["content"]` に抽出本文、`parsed["metadata"]` にメタデータが格納される。
#     → NLP 前処理へ供する際は、Unicode 正規化（NFC/NFKC）や改行規範の統一、機密情報の除去などを別途実施する。
#
# 注意:
#   - ネットワークに依存するため、オフライン環境や URL 変更時は失敗する。
#   - 大容量 PDF は解析時間・メモリを要する。実運用ではタイムアウトやサイズ上限、レート制御を設ける。
#   - ライセンス／著作権の確認は別途必要（抽出テキストの再配布・学習利用時）。

from tika import parser  # pip install tika

# 解析対象の PDF（ここでは GitHub Raw 上のサンプル）
pdf = "https://github.com/makaishi2/text-anl-samples/raw/master/pdf/sample.pdf"

# PDF を解析して本文とメタデータを取得する
# - 内部的には Java の Tika を起動し、HTTP 経由でファイル（または URL）を送って解析する
# - 戻り値: {"metadata": {…}, "content": "抽出テキスト", "status": 200, ...} といった辞書
parsed = parser.from_file(pdf)

# 補足（本例では出力しないが、典型的な利用方法）:
#   text = parsed.get("content", "")
#   meta = parsed.get("metadata", {})
#   print(text[:500])  # 先頭だけ確認
#   print(meta)

In [ ]:
# リスト2.1.11
# PDF読み込み結果の確認（Tika の解析結果 dict を JSON 体裁で整形表示）
# --------------------------------------------------------------------------------
# 目的:
#   - 直前の処理（parser.from_file(pdf)）で得られた `parsed`（辞書）を、可読性の高い
#     JSON 文字列に整形して標準出力へダンプする。
#
# 理論・実務メモ:
#   - `json.dumps(obj, ...)` は Python オブジェクトを JSON 文字列へ直列化する。
#     Tika の返り値は一般に { "content": str, "metadata": dict, "status": int, ... } という
#     JSON 化可能な基本型の集合なので、そのまま直列化できる。
#   - `indent=2` はインデント幅（2 スペース）を指定し、階層構造を視覚的に把握しやすくする。
#     これはデータ監査・EDA の初手として有用（キーの有無、型、想定外の空値などを発見しやすい）。
#   - `ensure_ascii=False` により、非 ASCII 文字（日本語など）を \uXXXX ではなく
#     そのままの文字で出力する。人間の目視確認に向く一方、端末のエンコーディングや
#     フォント設定が不適切だと文字化けする可能性はある。
#   - 解析対象が大きい PDF の場合、`parsed["content"]` は長大になり得る。
#     実務ではプレビュー（先頭 N 文字のみ表示）やファイル出力に切り替える方が効率的。
#     （ただし本スニペットは最小例として全量をそのまま表示している）
#   - 直列化エラーが出る典型ケースは「非 JSON 化可能な型」が含まれる場合だが、
#     Tika 標準の出力では通常発生しない。念のため `default=str` でフォールバックする
#     実装もあり得る（本例では挙動を変えないため採用しない）。
#
# 品質チェックの観点（コメントのみ）:
#   - 必須キーの存在: "content" が空で "status" が 200 以外なら抽出失敗の可能性。
#   - メタデータ: "Content-Type", "X-Parsed-By", "producer" などの有無を確認。
#   - 改行規範: 後段の NLP で扱いやすいよう、改行コードや空行の分布を把握する。

import json

# 解析結果 `parsed` を JSON 文字列へ整形し、Unicode を可読なまま出力
print(json.dumps(parsed, indent=2, ensure_ascii=False))

In [ ]:
# リスト 2.1.12
# タイトルの表示
# --------------------------------------------------------------------------------
# 目的:
#   - 直前の Tika 解析結果 `parsed` のメタデータから題名（タイトル）を取り出して標準出力に表示する。
#
# 理論・実務メモ（PDFメタデータの構造とTikaの振る舞い）:
#   - PDF の「タイトル」は複数経路で埋め込まれうる。
#       1) PDF Info 辞書の /Title（古典的メタ） → Tika では `pdf:docinfo:title` 等に反映されやすい
#       2) XMP（XMLベースの拡張メタ）内の Dublin Core → `dc:title`（言語別マップや配列をとりうる）
#       3) 一部ツールは単純化して `title` キーも併置する
#     → どのキーに格納されるかは作成ツール・埋め込み方針に依存するため **必ずしも 'title' が存在するとは限らない**。
#
#   - python-tika の返す `parsed["metadata"]` は「キー: 文字列 / 値: 文字列または配列（複数値）」の辞書。
#     同名メタが複数回現れるフォーマットでは **値が list** になることがある（例: `dc:title` が言語ごと配列）。
#     本スニペットは最小例のため、型や存在チェックは省略し、ダイレクトアクセスの挙動を示す。
#
# 例外と堅牢化に関する注意（本行では挙動は変えない、コメントのみ）:
#   - `parsed['metadata']` もしくは `['title']` が無い場合、`KeyError` が発生する。
#   - 実運用では `parsed.get('metadata', {}).get('title')` のように安全に取り出し、
#     それでも無い場合は `pdf:docinfo:title` や `dc:title` へフォールバックする決定規則を設けるのが理にかなう。
#   - `dc:title` が配列の場合は、優先言語（例: 'x-default' や 'ja'）を選ぶ規則を明示しておくと決定性が上がる。
#
# 出力の意味:
#   - ここで表示されるのは **PDF埋め込みメタデータのタイトル** であり、本文の最初の見出し等とは異なる。
#     教材・論文・報告書ではしばしば本文タイトルとメタタイトルが不一致になる点に注意。

print(parsed["metadata"]["title"])

In [ ]:
# リスト 2.1.13
# PDFコンテンツの表示（Tika 抽出テキストを改行除去して単一行で出力）
# --------------------------------------------------------------------------------
# 目的:
#   - `parsed["content"]`（Apache Tika による抽出本文）を、改行文字を除去して連結し、標準出力へ表示する。
#
# 理論・実務メモ:
#   - 改行の意味論:
#       * PDF はレイアウト保存型であり、「行末改行」は文法的な境界ではなくレイアウト由来であることが多い。
#         よって改行を無条件に消す（'' へ置換）と、単語や句が結合して読みづらくなる/トークンが誤結合する可能性がある。
#         代替として、解析用途では改行→空白（' '）や、段落改行のみ残すなどの規範化を設計するのが一般的。
#       * 下流の NLP（文分割・トークナイズ）では、段落・見出し・箇条書きの境界が特徴量となりうるため、
#         可視化の都合と解析の都合を分け、ここは「表示用」と割り切るのが妥当。
#   - 文字正規化:
#       * PDF 由来テキストには全角/半角・結合文字・異体字が混在しうる。必要に応じて NFC/NFKC を別段で適用する。
#   - 例外・堅牢化ポイント（本行では挙動を変えない）:
#       * `parsed['content']` が存在しない/None の場合、`replace` 呼び出しで AttributeError が発生する。
#         実運用では `text = (parsed.get('content') or '')` のようにフォールバックする。
#   - 出力サイズ:
#       * 大きな PDF では標準出力が膨大になる。EDA では先頭 N 文字のみ表示やファイル書き出しにするのが一般的。
#
# 注意: 本スニペットは「最小例」のため、上記の堅牢化や正規化はあえて実装していない（挙動は元コードと同一）。

print(parsed["content"].replace("\n", ""))